In [3]:
import pandas as pd
df = pd.read_csv("Growth For Impact Data Assignment_Data.csv")
df

,Company Name,Company Description,Unnamed: 2,Website URL,Linkedin URL,Careers Page URL,Job listings page URL,job post1 URL,job post1 title,job post2 URL,job post2 title,job post3 URL,job post3 title
0,Forest Stewardship Council,Promoting sustainable forestry to combat clima...,Example1-,https://fsc.org/en,https://www.linkedin.com/company/fsc-internati...,https://fsc.org/en/careers-at-fsc,https://fsc.org/en/careers-at-fsc,https://fsc.jobs.personio.com/job/2262148?lang...,Market Intelligence Manager (m-f-d),https://fsc.jobs.personio.com/job/2268639?lang...,Trademark Manager (m-f-d),https://fsc.jobs.personio.com/job/2262183?lang...,Value Chain Development Manager (m-f-d)
1,Polestar,Driving society forward with uncompromised ele...,Example2-,https://www.polestar.com/,https://www.linkedin.com/company/polestarcars/...,https://www.polestar.com/global/about/careers/,https://polestar.teamtailor.com/jobs,https://polestar.teamtailor.com/jobs/6551682-p...,"Planning, Ordering & Distribution Manager - Pa...",https://polestar.teamtailor.com/jobs/6538269-f...,Financial Accounting Manager - UK,https://polestar.teamtailor.com/jobs/6517579-r...,Retail Operation Manager
2,Sweep,Streamline carbon management and reduction acr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Climate Bonds Initiative,Mobilizing $100 trillion bond market for globa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fuse Energy,Accelerating global renewable energy transitio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Power Factors,Accelerating the energy transition with softwa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,Ecology Project International,Youth-driven conservation through local expert...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,Enverus,"Accelerating investment in efficient, renewabl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,FlexGen,Advanced energy storage solutions reducing env...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
'''Extracts WebsitE URL and linkedin URL for all the companies'''
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import requests

class WebsiteFinderSelenium:
    def __init__(self, headless=False):
        """Initialize Chrome driver with anti-detection measures"""
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        
        # Anti-detection measures
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        
        # Add realistic user agent
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
        
        self.driver = webdriver.Chrome(options=chrome_options)
        
        # Hide webdriver property
        self.driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """
                Object.defineProperty(navigator, 'webdriver', {
                    get: () => undefined
                })
            """
        })
        
        self.wait = WebDriverWait(self.driver, 10)
        self.search_count = 0
    
    def clean_url(self, url: str) -> str:
        """Clean and normalize URLs"""
        if not url:
            return None
        
        url = url.strip()
        
        # Remove common prefixes we don't want
        if url.startswith('›'):
            url = url[1:].strip()
        
        # Ensure https://
        if not url.startswith('http'):
            url = 'https://' + url
        
        # Remove trailing slashes
        url = url.rstrip('/')
        
        return url
    
    def validate_url(self, url: str) -> bool:
        """Quick check if URL is accessible"""
        try:
            response = requests.head(url, timeout=5, allow_redirects=True)
            return response.status_code < 400
        except:
            return False
    
    def extract_keywords(self, description: str, max_words: int = 3) -> str:
        """Extract key descriptive words from company description"""
        if not description:
            return ""
        
        # Common words to skip
        skip_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 
                     'of', 'with', 'by', 'from', 'up', 'about', 'into', 'through', 'during',
                     'including', 'our', 'we', 'your', 'their', 'is', 'are', 'was', 'were'}
        
        # Split and clean words
        words = description.lower().split()
        keywords = []
        
        for word in words:
            # Remove punctuation
            word = ''.join(c for c in word if c.isalnum())
            # Keep meaningful words
            if word and word not in skip_words and len(word) > 3:
                keywords.append(word)
                if len(keywords) >= max_words:
                    break
        
        return ' '.join(keywords)
    
    def search_google(self, company_name: str, description: str = "", search_type: str = "website") -> str:
        """Search Google using Selenium with company description
        
        Args:
            company_name: Name of the company
            description: Company description for context
            search_type: Either "website" or "linkedin"
        """
        try:
            # Go to Google
            self.driver.get("https://www.google.com")
            time.sleep(2)  # Increased delay
            
            # Accept cookies if popup appears
            try:
                accept_button = self.driver.find_element(By.XPATH, "//button[contains(., 'Accept') or contains(., 'I agree')]")
                accept_button.click()
                time.sleep(2)
            except:
                pass
            
            # Build search query based on search type
            if search_type == "linkedin":
                search_query = f"{company_name} linkedin"
                if description:
                    keywords = self.extract_keywords(description, max_words=2)
                    if keywords:
                        search_query = f"{company_name} {keywords} linkedin"
            else:  # website
                search_query = f"{company_name} official website"
                if description:
                    keywords = self.extract_keywords(description, max_words=3)
                    if keywords:
                        search_query = f"{company_name} {keywords} official website"
            
            print(f"    Search query: {search_query}")
            
            # Find search box and enter query
            search_box = self.driver.find_element(By.NAME, "q")
            search_box.send_keys(search_query)
            search_box.send_keys(Keys.RETURN)
            
            # Wait for results
            time.sleep(3)
            
            # Try to find the result URL based on search type
            try:
                if search_type == "linkedin":
                    # Look specifically for LinkedIn URLs
                    links = self.driver.find_elements(By.CSS_SELECTOR, "a[href*='linkedin.com/company']")
                    if links:
                        href = links[0].get_attribute('href')
                        # Clean LinkedIn URL (remove tracking parameters)
                        if '?' in href:
                            href = href.split('?')[0]
                        print(f"    Found LinkedIn: {href}")
                        return href
                    
                    # Fallback: check all links for linkedin
                    all_links = self.driver.find_elements(By.CSS_SELECTOR, "#search a")
                    for link in all_links[:10]:
                        href = link.get_attribute('href')
                        if href and 'linkedin.com/company' in href:
                            if '?' in href:
                                href = href.split('?')[0]
                            print(f"    Found LinkedIn via fallback: {href}")
                            return href
                
                else:  # website search
                    # Method 1: Look for cite elements
                    cite_elements = self.driver.find_elements(By.TAG_NAME, "cite")
                    for cite in cite_elements:
                        url_text = cite.text.strip()
                        if url_text and 'google.com' not in url_text.lower():
                            print(f"    Found via cite: {url_text}")
                            return self.clean_url(url_text)
                    
                    # Method 2: Look for result links
                    results = self.driver.find_elements(By.CSS_SELECTOR, "div.yuRUbf a")
                    if results:
                        href = results[0].get_attribute('href')
                        if href and 'google.com' not in href:
                            print(f"    Found via link: {href}")
                            return href
                    
                    # Method 3: Look for any visible link in results
                    links = self.driver.find_elements(By.CSS_SELECTOR, "#search a")
                    for link in links[:5]:
                        href = link.get_attribute('href')
                        if href and href.startswith('http') and 'google.com' not in href and 'youtube.com' not in href:
                            print(f"    Found via search link: {href}")
                            return href
                        
            except Exception as e:
                print(f"    Error extracting result: {e}")
            
            return None
            
        except Exception as e:
            print(f"    Search error: {e}")
            return None
    
    def process_csv(self, input_file: str, output_file: str, start_index: int = 0, batch_size: int = 20):
        """Process CSV file and find missing website URLs"""
        # Read CSV - use output file if it exists (to preserve previous work)
        import os
        if os.path.exists(output_file) and start_index > 0:
            print(f"Loading previous progress from: {output_file}")
            df = pd.read_csv(output_file)
        else:
            print(f"Loading original data from: {input_file}")
            df = pd.read_csv(input_file)
        
        # Process only a batch
        end_index = min(start_index + batch_size, len(df))
        
        print(f"Processing companies {start_index+1} to {end_index} (Total: {len(df)})...")
        
        # Track progress
        found_count = 0
        failed_companies = []
        
        for idx in range(start_index, end_index):
            row = df.iloc[idx]
            company_name = row['Company Name']
            website = row.get('Website URL', '')
            linkedin_url = row.get('Linkedin URL', '')
            description = row.get('Company Description', '')
            
            # Check if both website and LinkedIn already exist
            has_website = pd.notna(website) and str(website).strip() and str(website).strip() != 'nan'
            has_linkedin = pd.notna(linkedin_url) and str(linkedin_url).strip() and str(linkedin_url).strip() != 'nan'
            
            # Skip only if BOTH exist
            if has_website and has_linkedin:
                print(f"[{idx+1}/{len(df)}] {company_name}: ✓ Already has both website and LinkedIn")
                continue
            
            print(f"\n[{idx+1}/{len(df)}] Searching for: {company_name}")
            
            # Search for website if missing
            found_website = None
            if not has_website:
                print(f"    → Searching for website...")
                found_website = self.search_google(company_name, description, search_type="website")
                time.sleep(3)  # Delay between searches
            else:
                print(f"    ✓ Website already exists: {website}")
            
            # Search for LinkedIn URL if missing
            found_linkedin = None
            if not has_linkedin:
                print(f"    → Searching for LinkedIn...")
                found_linkedin = self.search_google(company_name, description, search_type="linkedin")
                time.sleep(3)  # Delay between searches
            else:
                print(f"    ✓ LinkedIn already exists: {linkedin_url}")
            
            # Process website results
            if found_website:
                # Validate URL
                print(f"    Validating website...")
                if self.validate_url(found_website):
                    df.at[idx, 'Website URL'] = found_website
                    found_count += 1
                    print(f"    ✓ Website saved: {found_website}")
                else:
                    # Try adding www
                    alt_url = found_website.replace('https://', 'https://www.')
                    if self.validate_url(alt_url):
                        df.at[idx, 'Website URL'] = alt_url
                        found_count += 1
                        print(f"    ✓ Website saved: {alt_url}")
                    else:
                        # Save it anyway, might just be slow to respond
                        df.at[idx, 'Website URL'] = found_website
                        found_count += 1
                        print(f"    ⚠ Website saved (couldn't validate): {found_website}")
            
            # Process LinkedIn results
            if found_linkedin:
                df.at[idx, 'Linkedin URL'] = found_linkedin
                print(f"    ✓ LinkedIn saved: {found_linkedin}")
            
            # Track failures
            if not found_website and not found_linkedin:
                print(f"    ✗ Could not find website or LinkedIn")
                failed_companies.append(company_name)
            elif not found_website:
                print(f"    ⚠ Could not find website (LinkedIn found)")
            elif not found_linkedin:
                print(f"    ⚠ Could not find LinkedIn (Website found)")
            
            # Save progress every 3 companies
            if (idx + 1) % 3 == 0:
                df.to_csv(output_file, index=False)
                print(f"\n--- Progress saved: {found_count} websites found ---")
            
            # Delay between companies (increased to avoid detection)
            time.sleep(8)  # Increased from 4 to 8 seconds
        
        # Final save
        df.to_csv(output_file, index=False)
        
        print("\n" + "="*60)
        print(f"BATCH COMPLETE!")
        print(f"Processed: {end_index - start_index} companies")
        print(f"Websites found: {found_count}")
        print(f"Failed: {len(failed_companies)}")
        
        if failed_companies:
            print("\nFailed companies:")
            for company in failed_companies[:10]:
                print(f"  - {company}")
        
        if end_index < len(df):
            print(f"\n⚠️  {len(df) - end_index} companies remaining.")
            print(f"To continue: Update START_INDEX = {end_index}")
        
        return df
    
    def close(self):
        """Close the browser"""
        self.driver.quit()


# USAGE EXAMPLE
if __name__ == "__main__":
    print("Initializing Chrome browser...")
    print("A Chrome window will open - don't close it!\n")
    
    finder = WebsiteFinderSelenium(headless=False)  # Set True to hide browser
    
    try:
        input_file = "Growth For Impact Data Assignment_Data.csv"
        output_file = "GFI_Data_Updated.csv"
        
        # Configure batch processing
        START_INDEX = 100   # Change to continue from where you left off
        BATCH_SIZE = 8   # Reduced to 10 to avoid CAPTCHA (was 20)
        
        print("Starting website URL finder with Selenium...")
        print("This uses real Chrome browser - more reliable!\n")
        
        df_updated = finder.process_csv(
            input_file, 
            output_file,
            start_index=START_INDEX,
            batch_size=BATCH_SIZE
        )
        
        print(f"\n✓ Results saved to: {output_file}")
        print("\nTo continue processing:")
        print(f"  1. Update START_INDEX = {START_INDEX + BATCH_SIZE}")
        print(f"  2. Run the script again")
        
    finally:
        print("\nClosing browser...")
        finder.close()
        print("Done!")

Initializing Chrome browser...
A Chrome window will open - don't close it!



2025-11-09 23:14:58,498 WARNING: There was an error managing chromedriver (error decoding response body); using driver found in the cache


Starting website URL finder with Selenium...
This uses real Chrome browser - more reliable!

Loading previous progress from: GFI_Data_Updated.csv
Processing companies 101 to 108 (Total: 173)...

[101/173] Searching for: Sightline Climate (CTVC)
    ✓ Website already exists: https://www.sightlineclimate.com
    → Searching for LinkedIn...
    Search query: Sightline Climate (CTVC) datadriven platform linkedin
    Found LinkedIn: https://www.linkedin.com/company/ctvc
    ✓ LinkedIn saved: https://www.linkedin.com/company/ctvc
    ⚠ Could not find website (LinkedIn found)

[102/173] Searching for: Pano AI
    ✓ Website already exists: https://www.pano.ai
    → Searching for LinkedIn...
    Search query: Pano AI revolutionizing disaster linkedin
    Found LinkedIn: https://www.linkedin.com/company/pano-ai
    ✓ LinkedIn saved: https://www.linkedin.com/company/pano-ai
    ⚠ Could not find website (LinkedIn found)

--- Progress saved: 0 websites found ---

[103/173] Searching for: Unirac
   

In [19]:
'''Extracts career page URL and job listings page URL'''
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
import os


class CareerPageFinder:
    def __init__(self, headless=False):
        """Initialize Chrome driver"""
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option("useAutomationExtension", False)

        # Initialize driver (make sure chromedriver is in PATH or use webdriver-manager)
        self.driver = webdriver.Chrome(options=chrome_options)
        # Hide webdriver flag
        try:
            self.driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
                "source": """
                    Object.defineProperty(navigator, 'webdriver', {
                        get: () => undefined
                    })
                """
            })
        except Exception:
            # not critical if CDP not available
            pass

        self.wait = WebDriverWait(self.driver, 10)

        # Known job board platforms and career keywords
        self.job_platforms = [
            'lever.co', 'greenhouse.io', 'workable.com', 'ashbyhq.com',
            'jobs.lever.co', 'boards.greenhouse.io', 'apply.workable.com',
            'zohorecruit.com', 'myworkdayjobs.com', 'successfactors.com',
            'icims.com', 'taleo.net', 'breezy.hr', 'personio.de',
            'personio.com', 'teamtailor.com', 'recruitee.com', 'bamboohr.com',
            'smartrecruiters.com', 'jobvite.com', 'fountain.com'
        ]

    def is_valid_url(self, url):
        """Check if URL is valid and accessible"""
        if not url or not isinstance(url, str):
            return False
        url = url.strip()
        if not (url.startswith('http://') or url.startswith('https://')):
            # try adding scheme
            url = 'https://' + url
        try:
            response = requests.head(url, timeout=6, allow_redirects=True)
            return response.status_code < 400
        except Exception:
            return False

    def find_career_page_from_website(self, website_url):
        """Find career page by crawling the website"""
        if not website_url or not isinstance(website_url, str):
            return None, None

        try:
            if not website_url.startswith('http'):
                website_url = 'https://' + website_url

            print(f"      Crawling website: {website_url}")
            self.driver.get(website_url)
            time.sleep(2)

            # Get all links on the page
            links = self.driver.find_elements(By.TAG_NAME, "a")

            career_candidates = []
            job_platform_links = []

            for link in links:
                try:
                    href = link.get_attribute('href')
                    text = (link.text or '').lower().strip()
                    if not href:
                        continue
                    href_lower = href.lower()

                    # Priority 1: Known job platforms
                    for platform in self.job_platforms:
                        if platform in href_lower:
                            job_platform_links.append((href, 10))
                            break

                    # Priority 2: Strong career keywords in URL
                    strong_keywords = ['careers', 'jobs', 'join-us', 'hiring', 'work-with-us', 'work-at', 'join']
                    for keyword in strong_keywords:
                        if f'/{keyword}' in href_lower or f'-{keyword}' in href_lower or f'/{keyword}/' in href_lower:
                            if any(word in text for word in ['career', 'job', 'join', 'work', 'hiring', 'opportunity']):
                                career_candidates.append((href, 8))
                            else:
                                career_candidates.append((href, 6))
                            break

                    # Priority 3: Career keywords in link text
                    if any(word in text for word in ['career', 'jobs', 'join us', 'work with us', 'opportunities', "we're hiring"]):
                        if href not in [c[0] for c in career_candidates]:
                            career_candidates.append((href, 5))
                except Exception:
                    continue

            # First, check job platform links
            if job_platform_links:
                best_link = max(job_platform_links, key=lambda x: x[1])[0]
                if self.is_valid_url(best_link):
                    print(f"      ✓ Found job platform: {best_link}")
                    jobs_listing = self.find_jobs_listing_page(best_link)
                    return best_link, jobs_listing

            # Then career candidates by priority
            if career_candidates:
                career_candidates.sort(key=lambda x: x[1], reverse=True)
                for candidate_url, priority in career_candidates:
                    if self.is_valid_url(candidate_url):
                        print(f"      ✓ Found career page: {candidate_url}")
                        jobs_listing = self.find_jobs_listing_page(candidate_url)
                        return candidate_url, jobs_listing

            # Try common patterns on base URL
            base_url = website_url.rstrip('/')
            common_paths = [
                '/careers', '/jobs', '/careers/', '/jobs/',
                '/work-with-us', '/join-us', '/join',
                '/about/careers', '/company/careers', '/team/careers'
            ]
            for path in common_paths:
                test_url = base_url + path
                if self.is_valid_url(test_url):
                    print(f"      ✓ Found career page via pattern: {test_url}")
                    jobs_listing = self.find_jobs_listing_page(test_url)
                    return test_url, jobs_listing

            print("      ✗ No career page found")
            return None, None

        except Exception as e:
            print(f"      Error crawling website: {e}")
            return None, None

    def find_jobs_listing_page(self, career_page_url):
        """Find the actual job listings page from career page"""
        if not career_page_url:
            return None

        try:
            # If on known job platform, try /jobs
            if any(platform in career_page_url for platform in self.job_platforms):
                print("      ✓ Career page is on job platform")
                base_platform_url = career_page_url.rstrip('/')
                if '/jobs' not in base_platform_url:
                    test_jobs_url = f"{base_platform_url}/jobs"
                    if self.is_valid_url(test_jobs_url):
                        print(f"      ✓ Found /jobs page: {test_jobs_url}")
                        return test_jobs_url
                return career_page_url

            # Load the career page
            if not career_page_url.startswith('http'):
                career_page_url = 'https://' + career_page_url
            self.driver.get(career_page_url)
            time.sleep(2)
            page_source = self.driver.page_source.lower()

            exclude_keywords = ['privacy', 'cookie', 'terms', 'contact', 'faq', 'news', 'events', 'blog']

            # Check for iframes (embedded job boards)
            iframes = self.driver.find_elements(By.TAG_NAME, "iframe")
            for iframe in iframes:
                try:
                    iframe_src = iframe.get_attribute('src') or ''
                    if iframe_src and any(platform in iframe_src for platform in self.job_platforms):
                        # avoid individual job postings (ending digits) and apply pages
                        if not re.search(r'/\d+$', iframe_src) and '/apply/' not in iframe_src.lower():
                            print(f"      ✓ Found job listing in iframe: {iframe_src}")
                            return iframe_src
                except Exception:
                    continue

            # Look for links on the career page
            links = self.driver.find_elements(By.TAG_NAME, "a")
            job_platform_candidates = []
            job_keyword_candidates = []
            redirect_candidates = []

            for link in links:
                try:
                    href = link.get_attribute('href')
                    text = (link.text or '').lower().strip()
                    if not href:
                        continue
                    href_lower = href.lower()

                    # Skip excluded pages
                    if any(excl in href_lower for excl in exclude_keywords):
                        continue

                    # Skip individual job postings (patterns like /1234, /job/123, /apply/123, /j/ABC)
                    if (
                        re.search(r'/\d+$', href_lower) or
                        '/apply/' in href_lower or
                        re.search(r'/job/\d+', href_lower) or
                        re.search(r'/j/[A-Za-z0-9\-]+', href_lower)
                    ):
                        continue

                    # Priority 1: Job platforms
                    for platform in self.job_platforms:
                        if platform in href_lower:
                            if '/jobs' in href_lower:
                                job_platform_candidates.append((href, 10))
                            else:
                                job_platform_candidates.append((href, 7))
                            break

                    # Priority 2: Strong job keywords in link text
                    strong_job_keywords = [
                        'job openings', 'view jobs', 'see jobs', 'open positions',
                        'view openings', 'see open roles', 'current openings', 'browse jobs'
                    ]
                    if any(keyword in text for keyword in strong_job_keywords):
                        redirect_candidates.append((href, text, 9))

                    # Priority 3: Generic job keywords
                    job_keywords = ['jobs', 'openings', 'positions', 'apply now', 'vacancies']
                    if any(keyword in text for keyword in job_keywords):
                        if '/jobs' in href_lower or '/careers' in href_lower or any(platform in href_lower for platform in self.job_platforms):
                            job_keyword_candidates.append((href, 8))
                except Exception:
                    continue

            # Follow redirect candidates first
            if redirect_candidates:
                redirect_candidates.sort(key=lambda x: x[2], reverse=True)
                for candidate_url, link_text, priority in redirect_candidates[:3]:
                    try:
                        print(f"      → Following link: '{link_text}' to {candidate_url}")
                        self.driver.get(candidate_url)
                        time.sleep(2)
                        final_url = self.driver.current_url or candidate_url

                        if any(platform in final_url for platform in self.job_platforms):
                            if '/jobs' not in final_url:
                                base_url = final_url.rstrip('/')
                                test_jobs_url = f"{base_url}/jobs"
                                if self.is_valid_url(test_jobs_url):
                                    print(f"      ✓ Found job listing page (redirect): {test_jobs_url}")
                                    return test_jobs_url
                            print(f"      ✓ Found job listing page (redirect): {final_url}")
                            return final_url

                        page_content = self.driver.page_source.lower()
                        if any(kw in page_content for kw in ['job title', 'job description', 'apply now', 'open position']):
                            print(f"      ✓ Found job listing page (redirect): {final_url}")
                            return final_url
                    except Exception:
                        continue

            # If job platform candidates exist, prefer them
            if job_platform_candidates:
                best = max(job_platform_candidates, key=lambda x: x[1])[0]
                if any(platform in best for platform in self.job_platforms) and '/jobs' not in best:
                    base_url = best.rstrip('/')
                    test_jobs_url = f"{base_url}/jobs"
                    if self.is_valid_url(test_jobs_url):
                        print(f"      ✓ Found job listing page: {test_jobs_url}")
                        return test_jobs_url
                if self.is_valid_url(best):
                    print(f"      ✓ Found job listing page: {best}")
                    return best

            # If keyword candidates exist, return best
            if job_keyword_candidates:
                best = max(job_keyword_candidates, key=lambda x: x[1])[0]
                if self.is_valid_url(best):
                    print(f"      ✓ Found job listing page: {best}")
                    return best

            # Heuristic: look for job-related content on the career page itself
            page_lower = page_source
            job_indicators = 0
            if 'apply now' in page_lower:
                job_indicators += 1
            if 'job title' in page_lower or 'position title' in page_lower:
                job_indicators += 2
            if 'job description' in page_lower:
                job_indicators += 1
            if 'open position' in page_lower or 'current opening' in page_lower:
                job_indicators += 1

            if job_indicators >= 2:
                print(f"      ✓ Career page contains job listings (indicators: {job_indicators})")
                return career_page_url

            # If on job platform but no /jobs found, try to append /jobs
            if any(platform in career_page_url for platform in self.job_platforms):
                base_url = career_page_url.rstrip('/')
                if '/jobs' not in base_url:
                    test_jobs_url = f"{base_url}/jobs"
                    if self.is_valid_url(test_jobs_url):
                        print(f"      ✓ Found job listing page (platform /jobs): {test_jobs_url}")
                        return test_jobs_url
                print("      ✓ Using career page as job listing page (on job platform)")
                return career_page_url

            # Fallback: return career page
            print("      ✓ Using career page as job listing page (fallback)")
            return career_page_url

        except Exception as e:
            print(f"      Error finding job listing page: {e}")
            return None

    def process_csv(self, input_file, output_file, start_index=0, batch_size=10):
        """Process CSV and find career pages and job listings"""
        # Load data (resume if output exists and start_index > 0)
        if os.path.exists(output_file) and start_index > 0:
            print(f"Loading previous progress from: {output_file}")
            df = pd.read_csv(output_file)
        else:
            print(f"Loading data from: {input_file}")
            df = pd.read_csv(input_file)

        # Ensure columns exist
        if 'Careers Page URL' not in df.columns:
            df['Careers Page URL'] = ''
        if 'Job listings page URL' not in df.columns:
            df['Job listings page URL'] = ''

        # Normalize types
        df['Careers Page URL'] = df['Careers Page URL'].astype(str)
        df['Job listings page URL'] = df['Job listings page URL'].astype(str)

        end_index = min(start_index + batch_size, len(df))
        print(f"Processing companies {start_index+1} to {end_index} (Total: {len(df)})...\n")

        found_count = 0

        for idx in range(start_index, end_index):
            row = df.iloc[idx]
            company_name = row.get('Company Name', f'Row {idx}')
            website_url = row.get('Website URL', '') or ''
            career_page = row.get('Careers Page URL', '')
            job_listing = row.get('Job listings page URL', '')

            # Skip if already filled
            has_career = pd.notna(career_page) and str(career_page).strip() not in ['', 'nan']
            has_listing = pd.notna(job_listing) and str(job_listing).strip() not in ['', 'nan']

            if has_career and has_listing:
                print(f"[{idx+1}/{len(df)}] {company_name}: ✓ Already has career and listing pages")
                continue

            print(f"\n[{idx+1}/{len(df)}] Processing: {company_name}")

            found_career = None
            found_listing = None

            # Try website only
            if pd.notna(website_url) and str(website_url).strip():
                found_career, found_listing = self.find_career_page_from_website(str(website_url).strip())

            if not found_career:
                print("      ✗ No career page found on company website - leaving blank")

            # Save results
            if found_career:
                df.at[idx, 'Careers Page URL'] = str(found_career)
                found_count += 1
                print("      ✓ Career page saved")
                if found_listing:
                    df.at[idx, 'Job listings page URL'] = str(found_listing)
                    print("      ✓ Job listing page saved")
            else:
                print("      ✗ No career/job pages found - leaving blank")

            # Periodic save
            if (idx + 1) % 3 == 0:
                df.to_csv(output_file, index=False)
                print(f"\n--- Progress saved: {found_count} career pages found ---")

            # polite delay
            time.sleep(3)

        # Final save
        df.to_csv(output_file, index=False)
        print("\n" + "=" * 60)
        print(f"BATCH COMPLETE!\nProcessed: {end_index - start_index} companies\nCareer pages found: {found_count}")
        if end_index < len(df):
            print(f"\n⚠️  {len(df) - end_index} companies remaining.\nTo continue: update START_INDEX = {end_index}")

        return df

    def close(self):
        """Close browser"""
        try:
            self.driver.quit()
        except Exception:
            pass


# === USAGE EXAMPLE ===
if __name__ == "__main__":
    print("Starting Career Page Finder...\n")

    finder = CareerPageFinder(headless=False)

    try:
        input_file = "gfi_data_updated.csv"      # your input CSV
        output_file = "GFI_Data_With_Careers.csv"  # output CSV to save progress

        START_INDEX = 147   # change to resume
        BATCH_SIZE = 25   # how many rows to process in this run

        df = finder.process_csv(input_file, output_file, START_INDEX, BATCH_SIZE)
        print(f"\n✓ Results saved to: {output_file}")

    finally:
        print("\nClosing browser...")
        finder.close()
        print("Done!")


Starting Career Page Finder...

Loading previous progress from: GFI_Data_With_Careers.csv
Processing companies 148 to 172 (Total: 173)...


[148/173] Processing: Context Labs
      Crawling website: https://contextlabs.com
      ✓ Found career page: https://contextlabs.com/company/careers/
      → Following link: 'explore open positions' to https://contextlabs.bamboohr.com/jobs/
      ✓ Found job listing page (redirect): https://contextlabs.bamboohr.com/careers
      ✓ Career page saved
      ✓ Job listing page saved

[149/173] Processing: 350.org
      Crawling website: https://350.org
      ✗ No career page found
      ✗ No career page found on company website - leaving blank
      ✗ No career/job pages found - leaving blank

[150/173] Processing: Scope3
      Crawling website: https://scope3.com › blog › category › company-news
      Error crawling website: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=142.0.7444.60)
Stacktrace:
Symbols not available. Du

In [108]:
'''Extracts Jobs and their details'''
import pandas as pd
import time
import os
from urllib.parse import urljoin
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# --- Parameters You Can Change ---

# Path to your input CSV file
CSV_FILE = 'GFI_Data_With_Careers.csv'

# Name of the column containing the LinkedIn URLs
LINKEDIN_COL = 'Linkedin URL'

# Name of the column with the company name (for logging)
COMPANY_COL = 'Company Name'

# Set your batch processing parameters
START_INDEX = 172  # Row to start from (0 is the first row)
BATCH_SIZE = 1   # Number of rows to process

# Define a single output file for all batches
OUTPUT_FILE = 'GFI_Data_With_Jobs_Scraped_1.csv'

# Define a path to store your LinkedIn profile/session
LINKEDIN_PROFILE_PATH = os.path.join(os.getcwd(), 'linkedin_profile')

# ---------------------------------

def setup_driver():
    """Sets up the Chrome WebDriver."""
    print("Setting up Chrome WebDriver...")
    # Set up Chrome options
    options = webdriver.ChromeOptions()
    
    # Add user-data-dir to save your login session
    options.add_argument(f"--user-data-dir={LINKEDIN_PROFILE_PATH}")
    
    options.add_argument("--start-maximized")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # Using webdriver-manager to automatically handle the driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def scrape_linkedin_jobs(driver, batch_df):
    """
    Scrapes job data from LinkedIn company pages.
    
    This function modifies a copy of the batch DataFrame and returns it.
    """
    
    # 1. Handle Login
    driver.get("https://www.linkedin.com/login")
    print("="*50)
    print("ACTION REQUIRED:")
    print("Please log in to your LinkedIn account in the browser window.")
    print("(If you are already logged in, just press Enter).")
    print("Your session will be saved for future runs.")
    print("="*50)
    input("Press Enter here to continue AFTER you are successfully logged in...")
    print("Login confirmed. Starting scraper...")
    
    # Create a copy of the batch to modify.
    batch_copy = batch_df.copy()
    
    # Get the batch of rows to process
    for index, row in batch_copy.iterrows():
        company_name = row.get(COMPANY_COL, f"Row {index}")
        linkedin_url = row.get(LINKEDIN_COL)
        
        print(f"\n--- Processing {company_name} ({linkedin_url}) ---")
        
        # 2. Validate URL
        if not isinstance(linkedin_url, str) or 'linkedin.com/company/' not in linkedin_url:
            print(f"Skipping: Invalid or missing LinkedIn URL.")
            continue
            
        # 3. Construct jobs URL and navigate
        # We add ?lang=en to force English, making selectors more consistent
        jobs_url = linkedin_url.rstrip('/') + '/jobs/?lang=en'
        
        try:
            driver.get(jobs_url)
            # Wait for the first job card to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "li[class*='job-card-container']"))
            )
            print("Jobs page loaded.")
            time.sleep(1) # Extra buffer for elements to render
            
        except TimeoutException:
            print("Skipping: Could not find job list (page timed out or no 'jobs' section).")
            continue
        except Exception as e:
            print(f"Skipping: Error loading jobs page: {e}")
            continue

        # 4. Find job list items
        try:
            # Scroll down to load more jobs (optional, but can help)
            driver.execute_script("window.scrollTo(0, 500);")
            time.sleep(1.5)

            job_elements = driver.find_elements(By.CSS_SELECTOR, "li[class*='job-card-container']")
            
            if not job_elements:
                print("No job postings found on the page.")
                continue
                
            print(f"Found {len(job_elements)} job postings. Extracting top 3...")
            
            # Loop by index (0, 1, 2) to prevent stale element errors
            job_limit = min(len(job_elements), 3)
            
            for job_index in range(job_limit):
                
                job_num = job_index + 1
                
                # Re-find the list in *every* loop iteration
                try:
                    all_job_elements = driver.find_elements(By.CSS_SELECTOR, "li[class*='job-card-container']")
                    if job_index >= len(all_job_elements):
                        print("  - Job list changed, stopping early.")
                        break
                    job_element = all_job_elements[job_index]
                except Exception as e:
                    print(f"  - Error re-finding job list: {e}. Skipping remaining jobs.")
                    break
                
                try:
                    # 6. CLICK the job item from the left list
                    job_element.click()
                    
                    # Wait for the description pane (right side) to update.
                    time.sleep(3.5) 
                    
                    # 7. EXTRACT ALL DETAILS from the right pane
                    job_title = "Not Found"
                    job_url = "Not Found"
                    job_location = "Not Found"
                    job_date = "Not Found"
                    job_description = "Not Found"
                    
                    # Get Title and URL from right pane H1 tag
                    try:
                        title_element = driver.find_element(By.CSS_SELECTOR, "h1[class*='t-24']")
                        job_title = title_element.text.strip()
                        try:
                            link_element = title_element.find_element(By.TAG_NAME, "a")
                            relative_url = link_element.get_attribute('href')
                            job_url = urljoin("https://www.linkedin.com", relative_url)
                        except NoSuchElementException:
                            print("  - Could not find job URL link inside title")
                    except NoSuchElementException:
                        print("  - Could not find job title (h1.t-24)")

                    # Get Location and Date from details container
                    try:
                        details_container = driver.find_element(By.CSS_SELECTOR, "div[class*='job-details-jobs-unified-top-card__tertiary-description']")
                        details_text = details_container.text
                        parts = details_text.split('·')
                        
                        if len(parts) > 0:
                            job_location = parts[0].strip()
                        if len(parts) > 1:
                            job_date = parts[1].strip()
                            
                    except NoSuchElementException:
                        print("  - Could not find details container (location/date)")
                    except Exception as e:
                        print(f"  - Error parsing location/date: {e}")

                    # Get description from its container (and handle "Show More")
                    try:
                        # 1. Find the new description container by its ID
                        description_container = driver.find_element(By.ID, "job-details")
                        
                        # 2. Try to find and click the "Show more" button
                        try:
                            # The button is *within* the "job-details" container's parent
                            article_container = driver.find_element(By.CLASS_NAME, "jobs-description__container")
                            show_more_button = article_container.find_element(By.CSS_SELECTOR, "button[class*='description__show-more-less-button'][aria-expanded='false']")
                            
                            driver.execute_script("arguments[0].click();", show_more_button)
                            print("  - Clicked 'Show more' for description.")
                            time.sleep(1.0) # Wait for text to expand
                        except NoSuchElementException:
                            pass # Button not found, description is likely already expanded
                        except Exception as e:
                            print(f"  - Note: Error clicking 'Show more' (might not exist): {e}")
                            
                        # 3. Get the description text
                        # Use .get_attribute('innerText') for robustness
                        job_description = description_container.get_attribute('innerText').strip()
                        
                    except NoSuchElementException:
                        print(f"  - Could not find job description container (ID='job-details') for '{job_title}'")
                    
                    # 8. Store results in the DataFrame copy
                    batch_copy.at[index, f'job post{job_num} title'] = job_title
                    batch_copy.at[index, f'job post{job_num} URL'] = job_url
                    batch_copy.at[index, f'job post{job_num} location'] = job_location
                    batch_copy.at[index, f'job post{job_num} date'] = job_date
                    batch_copy.at[index, f'job post{job_num} description'] = job_description
                    
                    print(f"  + Extracted job {job_num}: {job_title}")
                    
                except StaleElementReferenceException:
                    print("  - StaleElementReferenceException occurred. Skipping this job.")
                    continue
                except Exception as e:
                    print(f"  - Error extracting details for one job: {e}")
                    # Try to recover by scrolling and waiting
                    driver.execute_script("window.scrollBy(0, 100);")
                    time.sleep(1)

        except Exception as e:
            print(f"Error processing job list for {company_name}: {e}")

    # Return the modified batch DataFrame
    return batch_copy

def main():
    
    # --- Load Data ---
    # **NEW FIX: Check if the output file already exists to preserve old data**
    if os.path.exists(OUTPUT_FILE):
        print(f"Found existing output file. Loading from: {OUTPUT_FILE}")
        input_file_to_load = OUTPUT_FILE
    else:
        print(f"No existing output file found. Loading from original: {CSV_FILE}")
        input_file_to_load = CSV_FILE
        
    try:
        # Load the dataframe from the correct file (either original or previous output)
        df = pd.read_csv(input_file_to_load)
    except FileNotFoundError:
        print(f"ERROR: Input file not found: {input_file_to_load}")
        return
    except Exception as e:
        print(f"ERROR: Could not read CSV: {e}")
        return
        
    print(f"Loaded {len(df)} rows from {input_file_to_load}.")
    
    # Calculate end index
    end_index = min(START_INDEX + BATCH_SIZE, len(df))
    print(f"Processing batch: rows {START_INDEX} to {end_index - 1}")

    if START_INDEX >= len(df):
        print("Start index is beyond the file length. Nothing to process.")
        return
        
    # Select the batch of rows to process
    batch_df = df.iloc[START_INDEX:end_index]
    
    # --- Run Scraper ---
    driver = None
    if batch_df.empty:
        print("Batch is empty. Nothing to process.")
        return
        
    try:
        driver = setup_driver()
        # Scraper returns the processed batch
        scraped_data_df = scrape_linkedin_jobs(driver, batch_df)
        
        if driver:
            driver.quit()
        
        # --- Save Results ---
        if scraped_data_df.empty:
            print("No data was scraped in this batch.")
            return

        # Ensure new columns from the scraped batch exist in the main DataFrame
        new_cols = [col for col in scraped_data_df.columns if col not in df.columns]
        
        if new_cols:
            print(f"Adding new columns to main DataFrame: {new_cols}")
            for col in new_cols:
                df[col] = pd.NA
        
        # Update the main DataFrame with the new data from the batch
        df.update(scraped_data_df)
        
        # Save the *entire* updated DataFrame
        df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8')
        print(f"\nScraping complete for batch. {len(scraped_data_df)} rows processed.")
        print(f"All data saved to {OUTPUT_File}")
        
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
    finally:
        if driver:
            print("Cleaning up WebDriver...")
            driver.quit()

if __name__ == "__main__":
    # To run this in Jupyter, you would copy the contents of main()
    # and the helper functions into cells and then call main() in the last cell.
    main()

Ensure you have run: !pip install pandas selenium webdriver-manager
Found existing output file. Loading from: GFI_Data_With_Jobs_Scraped_1.csv
Loaded 173 rows from GFI_Data_With_Jobs_Scraped_1.csv.
Processing batch: rows 172 to 172
Setting up Chrome WebDriver...
ACTION REQUIRED:
Please log in to your LinkedIn account in the browser window.
(If you are already logged in, just press Enter).
Your session will be saved for future runs.


Press Enter here to continue AFTER you are successfully logged in... 


Login confirmed. Starting scraper...

--- Processing The Aspen Institute (nan) ---
Skipping: Invalid or missing LinkedIn URL.

Scraping complete for batch. 1 rows processed.

An unexpected error occurred: name 'OUTPUT_File' is not defined
Cleaning up WebDriver...


In [109]:
import pandas as pd
import numpy as np
import os

OUTPUT_FILE = 'GFI_Data_With_Jobs_Scraped_1.csv'

if not os.path.exists(OUTPUT_FILE):
    print(f"Output file not found: {OUTPUT_FILE}")
    print("Please run the scraper script at least once.")
else:
    try:
        # Load the saved data
        df = pd.read_csv(OUTPUT_FILE)
        
        # Define the job title columns
        job_cols = ['job post1 title', 'job post2 title', 'job post3 title']
        
        total_jobs_extracted = 0
        
        print("--- Job Extraction Report ---")
        
        for col in job_cols:
            if col in df.columns:
                # Count where the column is not null AND not 'Not Found'
                count = df[
                    (df[col].notna()) & (df[col] != 'Not Found')
                ].shape[0]
                
                print(f"Jobs found for '{col}': {count}")
                total_jobs_extracted += count
            else:
                print(f"Column '{col}' not found in the file yet.")
                
        print("-----------------------------")
        print(f"Total jobs extracted: {total_jobs_extracted}")

    except Exception as e:
        print(f"An error occurred while reading the file: {e}")

--- Job Extraction Report ---
Jobs found for 'job post1 title': 118
Jobs found for 'job post2 title': 107
Jobs found for 'job post3 title': 104
-----------------------------
Total jobs extracted: 329


In [111]:
import pandas as pd
import os

# The final output file, which contains all data
OUTPUT_FILE = 'GFI_Data_With_Jobs_Scraped_1.csv'
# The input file, as a fallback
INPUT_FILE = 'GFI_Data_With_Careers.csv'

# Determine which file to load
file_to_load = ""
if os.path.exists(OUTPUT_FILE):
    file_to_load = OUTPUT_FILE
    print(f"Loading data from your most recent output file: {OUTPUT_FILE}")
elif os.path.exists(INPUT_FILE):
    file_to_load = INPUT_FILE
    print(f"Loading data from your original input file: {INPUT_FILE}")
else:
    file_to_load = None
    print(f"ERROR: Neither {OUTPUT_FILE} nor {INPUT_FILE} could be found.")

if file_to_load:
    try:
        df = pd.read_csv(file_to_load)
        
        # Get total rows for context
        total_rows = len(df)
        print(f"Loaded {total_rows} total company rows.")
        print("--- URL Extraction Report ---")
        
        # List of columns to check
        # (I noticed you typed "linkedin urls" twice, 
        # so I'm assuming you meant all the main URL columns)
        url_columns = [
            'Website URL',
            'Linkedin URL',
            'Careers Page URL',
            'Job listings page URL'
        ]
        
        for col in url_columns:
            if col in df.columns:
                # Count non-null (notna) values. This is the most reliable way.
                count = df[col].notna().sum()
                print(f"Total '{col}': {count} (out of {total_rows})")
            else:
                print(f"Column '{col}' not found in the file.")
                
        print("-----------------------------")

    except Exception as e:
        print(f"An error occurred while reading the file: {e}")

Loading data from your most recent output file: GFI_Data_With_Jobs_Scraped_1.csv
Loaded 173 total company rows.
--- URL Extraction Report ---
Total 'Website URL': 170 (out of 173)
Total 'Linkedin URL': 161 (out of 173)
Total 'Careers Page URL': 109 (out of 173)
Total 'Job listings page URL': 106 (out of 173)
-----------------------------


# Growth For Impact - Web Scraping Assignment

## Overview
This project implements a multi-stage web scraping pipeline to extract comprehensive job posting information from climate-focused companies. Starting with only company names and descriptions, the system progressively gathers online presence data, career pages, and detailed job listings.

---

## Methodology

### Stage 1: Website & LinkedIn URL Extraction

**Objective:** Find official company websites and LinkedIn profiles using company name and description.

**Implementation:** `WebsiteFinderSelenium` class

**Process:**
1. **Search Query Construction**
   - Extracts 2-3 key descriptive words from company description
   - Filters out common stop words (the, a, and, etc.)
   - Creates targeted Google search queries combining company name with keywords
   - Example: `"Polestar driving society uncompromised official website"`

2. **Intelligent Web Search**
   - Uses Selenium WebDriver to perform real Google searches
   - Implements anti-detection measures:
     - Custom user agents
     - Disabled automation flags
     - Realistic browser behavior with delays
   - Separate searches for website and LinkedIn URLs

3. **Result Extraction**
   - Parses multiple HTML elements (cite tags, result links, search links)
   - Validates URLs by checking accessibility
   - Attempts alternative formats (www prefix) if initial validation fails

4. **Progressive Processing**
   - Batch processing (8 companies at a time) to avoid CAPTCHA
   - Saves progress every 3 companies
   - 8-second delays between searches to mimic human behavior
   - Resumes from last checkpoint if interrupted

**Key Features:**
- Context-aware searching using company descriptions
- Robust URL validation and normalization
- Session persistence for interrupted runs
- Rate limiting to avoid detection

---

### Stage 2: Career Page & Job Listings URL Extraction

**Objective:** Locate career pages and job listing portals from company websites.

**Implementation:** `CareerPageFinder` class

**Process:**
1. **Website Crawling**
   - Navigates to company website using Selenium
   - Extracts all links from the page
   - Handles cookie consent popups automatically

2. **Multi-Tier Detection System**

   **Priority 1 - Known Job Platforms (Score: 10)**
   - Detects 20+ ATS platforms: Lever, Greenhouse, Workable, Ashby, BambooHR, etc.
   - Automatically identifies embedded job boards

   **Priority 2 - Strong Career Keywords in URL (Score: 6-8)**
   - URL patterns: `/careers`, `/jobs`, `/join-us`, `/hiring`, `/work-with-us`
   - Cross-validates with link text content
   
   **Priority 3 - Link Text Analysis (Score: 5)**
   - Searches for phrases: "career", "jobs", "join us", "work with us", "opportunities"

3. **Job Listings Page Discovery**
   - Follows redirect links with phrases like "view jobs", "open positions"
   - Detects embedded iframes containing job boards
   - Checks for job listing indicators in page content:
     - "apply now", "job title", "job description", "open position"
   - Attempts `/jobs` suffix on job platform URLs
   - Falls back to career page if job-specific page not found

4. **URL Filtering**
   - Excludes individual job postings (patterns like `/job/1234`, `/apply/123`)
   - Filters out non-career pages (privacy, terms, blog, news)
   - Validates all URLs before saving

**Key Features:**
- Hierarchical scoring system for link relevance
- Intelligent redirect following
- Iframe detection for embedded job boards
- Content-based heuristics for validation
- Common URL pattern testing

---

### Stage 3: Job Details Extraction from LinkedIn

**Objective:** Scrape detailed job information from LinkedIn company pages.

**Implementation:** Main scraping function with Selenium

**Process:**
1. **Authentication & Session Management**
   - One-time manual LinkedIn login
   - Session cookies saved in `linkedin_profile` directory
   - Automatic session reuse for subsequent runs
   - User-data-dir profile persistence

2. **Jobs Page Navigation**
   - Constructs jobs URL: `{linkedin_company_url}/jobs/?lang=en`
   - Forces English language for consistent selectors
   - Waits for job card elements to load (10-second timeout)
   - Performs scroll actions to trigger lazy-loaded content

3. **Job Card Iteration**
   - Identifies job cards using CSS selector: `li[class*='job-card-container']`
   - Processes top 3 jobs per company
   - Re-finds job list in each iteration to prevent stale element errors
   - Clicks each job card to load details in right pane

4. **Detailed Information Extraction**
   
   For each job, extracts:
   
   **Job Title & URL**
   - Selector: `h1[class*='t-24']`
   - Extracts embedded link using `urljoin` for absolute URLs
   
   **Location & Date Posted**
   - Selector: `div[class*='job-details-jobs-unified-top-card__tertiary-description']`
   - Parses text split by '·' separator
   
   **Job Description**
   - Selector: `#job-details`
   - Clicks "Show more" button if present: `button[class*='description__show-more-less-button'][aria-expanded='false']`
   - Extracts full expanded text using `innerText` attribute

5. **Data Storage Structure**
   ```
   job post1 title, job post1 URL, job post1 location, job post1 date, job post1 description
   job post2 title, job post2 URL, job post2 location, job post2 date, job post2 description
   job post3 title, job post3 URL, job post3 location, job post3 date, job post3 description
   ```

6. **Error Handling**
   - `StaleElementReferenceException`: Re-finds elements and continues
   - `TimeoutException`: Skips company if jobs page doesn't load
   - Missing elements: Records "Not Found" for specific fields
   - Recovers from errors with scroll actions and wait periods

**Key Features:**
- Session persistence eliminates repeated logins
- Robust element re-finding strategy
- Dynamic content expansion (Show more buttons)
- Comprehensive error recovery
- Batch processing with progress preservation

---

## Technical Architecture

### Technologies Used
- **Python 3.12**
- **Selenium WebDriver** - Browser automation
- **Pandas** - Data manipulation and CSV handling
- **BeautifulSoup4** - HTML parsing (auxiliary)
- **webdriver-manager** - Automatic ChromeDriver management
- **Requests** - URL validation

### Anti-Detection Measures
- Randomized user agents
- Disabled automation flags
- CDP commands to hide webdriver property
- Human-like delays (3-8 seconds between actions)
- Batch processing to avoid rate limits
- Session persistence to reduce suspicious activity

### Data Persistence Strategy
- **Single Output File Approach**: All batches update the same CSV
- **Incremental Updates**: Only processes missing data
- **Checkpoint System**: Saves every 3 companies
- **Resume Capability**: Continues from `START_INDEX` parameter
- **Data Validation**: Checks for existing data before scraping

---

# Run main() - will prompt for LinkedIn login on first run
```

### Data Quality Features
- Validated URLs (accessibility checked)
- Cleaned and normalized data
- Comprehensive error handling
- "Not Found" placeholders for missing data
- UTF-8 encoding support

